In [1]:
## OGM
using Mosek, JuMP, OffsetArrays, LinearAlgebra , MosekTools, Gurobi ,Ipopt, KNITRO

include("code_to_compute_pivoted_cholesky.jl")


## elementary operations

function e_i(n, i)
    e_i_vec = zeros(n, 1)
    e_i_vec[i] = 1
    return e_i_vec
end

function ⊙(a,b)
    return ((a*b') .+ transpose(a*b')) ./ 2
end

function compute_rank(X, ϵ_sparsity)
    eigval_array_X = eigvals(X)
    rnk_X = 0
    n = length(eigval_array_X)
    for i in 1:n
        if abs(eigval_array_X[i]) >= ϵ_sparsity
            rnk_X = rnk_X + 1
        end
    end
    return rnk_X
end


## stepsize of proximal gradient method

function construct_GD_stepsizes(N)
    α = OffsetArray(zeros(N, N), 1:N, 0:N-1)

    for i in 1:N
        for j in 0:i-1
            α[i,j]=1
        end
    end
     
    return α   
end




function construct_FISTA_stepsizes(N; fista_type =:complex) #simple, complex
    α = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    β = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    ϕ = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    ψ = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    
    θ = OffsetArray(zeros(N+1),0:N) 
    θ[0] = 1
    for i in 1:N
        θ[i] = (1+sqrt(1+4*θ[i-1]^2))/2
    end
    
    ζ = OffsetVector(zeros(N), 0:N-1)
    
    if fista_type==:complex
        for i in 0:N-1
            ζ[i] =(θ[i]-1)/θ[i+1]
        end
        
    elseif fista_type==:simple
        for i in 0:N-1
            ζ[i] =(i)/(i+3)
        end
    end
    
    if N!=1
        mod = Model(Gurobi.Optimizer)
 
        @variable(mod, a[1:N, 0:N-1])
        @variable(mod, b[1:N, 0:N-1])
 
        for i in 0:N-1
           for j in 0:N-1
              if j > i
                 fix(a[i+1,j], 0; force = true)
                 fix(b[i+1,j], 0; force = true)
              end
           end
           fix(b[i+1,i],1; force=true)
           fix(a[i+1,i],1+ζ[i]; force=true)
        end
        
        for i in 1:N-1
            for j in 0:i-1
                @constraint(mod, b[i+1,j].==a[i,j])
            end
        end
        
        
        for i in 1:N-1
            for j in 0:i-1
                 @constraint(mod, b[i+1,j] + ζ[i]*b[i+1,j] - ζ[i]*b[i,j] .== a[i+1,j] )
            end
        end
          
        @objective(mod, Min, 0)
        
        optimize!(mod)
 
        if termination_status(mod) != OPTIMAL
           @error "termination status is not optimal"
        end
    
        solution_a=value.(a)
        solution_b=value.(b)
            
        for i in 0:N-1
            for j in 0:N-1
                α[i+1,j]=solution_a[i+1,j]
                β[i+1,j]=solution_a[i+1,j]
                ψ[i+1,j]=solution_b[i+1,j]
                ϕ[i+1,j]=solution_b[i+1,j]
            end
        end
            
    end
    
    if N==1 
        α[1,0]=1
        β[1,0]=1
        ψ[1,0]=1
        ϕ[1,0]=1
    end

    return ϕ,ψ,α,β
    
end


function get_theta(N)
    theta=OffsetArray(zeros(N+1), 0:N)
    theta[0]=1
    for i in 1:N-1
        theta[i]=(1+sqrt(1+4*theta[i-1]^2))/2
    end
    theta[N]=(1+sqrt(1+8*theta[N-1]^2))/2
    return theta
end

function get_lambda_OGM(N)
    rr=get_theta(N)
    rrr=OffsetArray(zeros(N),1:N)
    for i in 1:N
        rrr[i]=(2*rr[i-1]^2)/(rr[N])^2
    end
    return rrr
end

function get_tau_OGM(N) 
    rr=get_theta(N)
    rrr=OffsetArray(zeros(N+1),0:N)
    for i in 1:N
        rrr[i-1]=(2*rr[i-1])/(rr[N])^2
    end
    rrr[N]=1/(rr[N])
    return rrr
end


function new_ogm(N) #same function with construct_OGM_stepsizes
    theta=get_theta(N)
    α=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    for i in 0:N-1
        for k in 0:i-1
            if k+1<=i
                α[i+1,k]=α[k+1,k]+sum(2*theta[k]/theta[l+1]-(1/theta[l+1])*α[l,k] for l in k+1:i)
            end
        end
        α[i+1,i]=1+(2*theta[i]-1)/theta[i+1]
    end
    return α
end

## data generations

function data_generator_function(N, α, L)

    dim_𝐰 = N+2
    dim_𝐟′ = N+2
    dim_𝐟 = N+1
    
    𝐰_0 = zeros(dim_𝐰, 1) #optimum
    𝐰_star = e_i(dim_𝐰, N+2) #starting point

    #create gradients
    
    𝐟′ = OffsetArray(zeros(dim_𝐟′, N+2), 1:dim_𝐟′, -1:N)  

    # fill the elements of 𝐟′

    for i in 0:N
        𝐟′[:,i] = e_i(dim_𝐟′, i+1)
    end

    #create function evaluations

    𝐟 = OffsetArray(zeros(dim_𝐟, N+2), 1:dim_𝐟, -1:N)

    # fill elements of 𝐟

    
    for i in 0:N
        𝐟[:,i] = e_i(dim_𝐟, i+1)
    end

    #make 𝐰
    
    𝐰 = OffsetArray(Matrix{Any}(undef, dim_𝐰, N+2), 1:dim_𝐰, -1:N)

     𝐰[:,-1] = 𝐰_star

     𝐰[:, 0] = 𝐰_0
    
    for i in 0:N-1
        𝐰[:, i+1] = 𝐰[:,0] - (1/L)*sum(α[i+1,j]*𝐟′[:,j] for j in 0:i) 
    end
    
    return 𝐰, 𝐟′, 𝐟

end


##define matrices

function A_mat_f(i, j, α, 𝐟′, 𝐰)  
    return ⊙(𝐟′[:,j], 𝐰[:,i] - 𝐰[:,j])
end


function B_mat(i, j, α, 𝐰)     
    return ⊙(𝐰[:,i] - 𝐰[:,j], 𝐰[:,i] - 𝐰[:,j])
end

function C_mat_f(i, j, 𝐟′)
    return ⊙(𝐟′[:,i]-𝐟′[:,j], 𝐟′[:,i]-𝐟′[:,j])
end

function D_mat_f(i, j, 𝐟′)
    return ⊙(𝐟′[:,i], 𝐟′[:,j])
end

function E_mat_f(i, j, 𝐟′,𝐰)
    return ⊙(𝐰[:,i], 𝐟′[:,j])
end

function F_mat_f(i, j ,𝐰)
    return ⊙(𝐰[:,i], 𝐰[:,j])
end


function a_vec_f(i, j, 𝐟)
    return 𝐟[:,j]-𝐟[:,i]
end


## indices

struct i_j_idx 
    i::Int64 
    j::Int64 
end


function index_set_constructor(N)

    # construct idx_f
    idx_f = [i for i in -1:N]

    # construct the index set for λ
    idx_set_λ = i_j_idx[]
    for i in idx_f
        for j in idx_f
            if i!=j
                push!(idx_set_λ, i_j_idx(i,j))
            end
        end
    end
    
    return idx_set_λ
end

function effective_index_set_finder(λ, ϵ_tol)

    idx_set_λ_current = (λ.axes)[1]
    idx_set_λ_effective = i_j_idx[]

    for i_j_λ in idx_set_λ_current
        if abs(λ[i_j_λ]) >= ϵ_tol 
            push!(idx_set_λ_effective, i_j_λ)
        end
    end
    
    return idx_set_λ_effective

end

function zero_finder_cholesky(L_cholesky; ϵ_tol = ϵ_tol_Cholesky)
    dim_cholesky, _ = size(L_cholesky)
    zero_idx_set_L_cholesky = []
    for i in 1:dim_cholesky
        for j in 1:dim_cholesky
            if i >= j 
                if abs(L_cholesky[i,j]) <= ϵ_tol
                    push!(zero_idx_set_L_cholesky, (i,j))
                end
            end
        end
    end
    return zero_idx_set_L_cholesky
end



## primal
function solve_inner_primal(N, L, α, R; show_output = :off)

    𝐰, 𝐟′, 𝐟 = data_generator_function(N, α, L)

    dim_G = N+2
    dim_F = N+1
    idx_set_λ = index_set_constructor(N)
    
    #model
    model_primal = Model(optimizer_with_attributes(Mosek.Optimizer))

    #variables

    @variable(model_primal, F[1:dim_F])
    @variable(model_primal, G[1:dim_G, 1:dim_G], PSD)

    # objective

    @objective(model_primal, Max, F'*(a_vec_f(-1, N, 𝐟)) )

    # constraints

    @constraint(model_primal, tr(G*B_mat(0, -1, α, 𝐰)) <= R^2)

    for i_j_λ in idx_set_λ
        @constraint(model_primal,  
        F'*a_vec_f(i_j_λ.i, i_j_λ.j, 𝐟) + tr(G*( A_mat_f(i_j_λ.i, i_j_λ.j, α, 𝐟′, 𝐰)
            + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) ) <= 0 )
    end

#     for i_j_λ in idx_set_λ
#         @constraint(model_primal,  
#         tr(G*D_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) == 0 )
#     end
    

        
    #optimize

    if show_output == :off
        set_silent(model_primal)
    end

    optimize!(model_primal)
    
    if termination_status(model_primal) != MOI.OPTIMAL
        @info "💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀"
        @error "primal did not reach optimality;  termination status = " termination_status(model_primal)
    end
    
    # store and return 
    obj_val_primal = objective_value(model_primal)
    G_star = value.(G)
    F_star = value.(F)

    @info "performance measure of primal is $(obj_val_primal)"
    return obj_val_primal, G_star, F_star

end



solve_inner_primal (generic function with 1 method)

In [2]:
α=new_ogm(3)

3×3 OffsetArray(::Matrix{Float64}, 1:3, 0:2) with eltype Float64 with indices 1:3×0:2:
 1.61803  0.0      0.0
 1.79217  2.01939  0.0
 1.84923  2.35345  1.92996

In [3]:
N=3
L=1
R=1

α=new_ogm(N)
#α=construct_GD_stepsizes(N)

obj_val_primal, G1, F1= solve_inner_primal(N, L, α, R; show_output = :off)

┌ Info: performance measure of primal is 0.03769241682280323
└ @ Main In[1]:378


(0.03769241682280323, [0.35675967453551155 0.021586937840845123 … -0.019919184797234084 -0.5013257120528977; 0.021586937840845123 0.09296212317907745 … -0.02263502755455483 -0.1661529650209758; … ; -0.019919184797234084 -0.02263502755455483 … 0.035741751420328455 0.07515424823516281; -0.5013257120528977 -0.1661529650209758 … 0.07515424823516281 0.9999999095006444], [0.32294586857545066, 0.08474350777744455, 0.04802323757405689, 0.03769241682280323])

In [4]:
for i in 1:15
    θ=get_theta(i)
    print(1/(2*(θ[i]^2-1)))
end

0.166666666666666660.070638393637995010.040765495722153680.0269636130788651650.0193058564602343920.014568426149731570.0114152139815515850.0092023863544976240.00758571991215044060.0063665247100409560.0054231333673952050.0046774487748713980.0040773718202308790.00358701127189078740.003180958141197731

In [5]:
(2*θ[0]-1)/(2*θ[1])

LoadError: UndefVarError: θ not defined

In [6]:
v = eigvecs(G1)[:,1]

5-element Vector{Float64}:
 0.28873296358121814
 0.46717762059745116
 0.6331504309109309
 0.525982247886214
 0.14436603929033573

In [7]:
G= G2-G1

LoadError: UndefVarError: G2 not defined

In [8]:
G3

LoadError: UndefVarError: G3 not defined

In [9]:
D = OffsetArray(zeros(N+2, N+2), 1:N+2, 1:N+2)
for i in 1:N+2
    for j in 1:N+2
        if i!=j
            D[i,j]=0
        else
            D[i,j]=G[i,j]
        end
    end
end

LoadError: UndefVarError: G not defined

In [10]:
D*v

5-element OffsetArray(::Vector{Float64}, 1:5) with eltype Float64 with indices 1:5:
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
G1

5×5 Matrix{Float64}:
  0.35676     0.0215869  -0.0477638  -0.0199192  -0.501326
  0.0215869   0.0929621  -0.0217487  -0.022635   -0.166153
 -0.0477638  -0.0217487   0.041454   -0.0210429   0.0607697
 -0.0199192  -0.022635   -0.0210429   0.0357418   0.0751542
 -0.501326   -0.166153    0.0607697   0.0751542   1.0

In [12]:
A_mat_f(0,1, α, 𝐟′, 𝐰)

LoadError: UndefVarError: 𝐟′ not defined

In [13]:
eigvals(G)

LoadError: UndefVarError: G not defined

In [14]:
eigvecs(G2)

LoadError: UndefVarError: G2 not defined

In [15]:
G_star

LoadError: UndefVarError: G_star not defined

In [16]:
G_star[1,1]=10

LoadError: UndefVarError: G_star not defined

In [17]:
eigvals(G_star)

LoadError: UndefVarError: G_star not defined

In [18]:
𝐰, 𝐟′, 𝐟 = data_generator_function(N,α, L) 

grad=OffsetArray(zeros(N+1), 0:N)
coeff=OffsetArray(zeros(N+1), 0:N)
func=OffsetArray(zeros(N+1), 0:N)
for j in 0:N
    grad[j]=tr(G_star*D_mat_f(j, j, 𝐟′))
    coeff[j]=tr(G_star*E_mat_f(-1, j, 𝐟′,𝐰))
    func[j]=F_star'𝐟[:,j]
end

alpha_lower = OffsetArray(zeros(N, N), 1:N, 0:N-1)

for i in 1:N
    for j in 0:i-1
        alpha_lower[i,j]= (func[j]-func[i]+(1/(2*L))*grad[i]+(1/(2*L))*grad[j])/((1/L)*grad[j])
    end
end

LoadError: UndefVarError: G_star not defined

In [19]:
grad

4-element OffsetArray(::Vector{Float64}, 0:3) with eltype Float64 with indices 0:3:
 0.0
 0.0
 0.0
 0.0

In [457]:
D1 = D_mat_f(2, 4, 𝐟′)
D2 = D_mat_f(1, 4, 𝐟′)
D3 = D_mat_f(3, 4, 𝐟′)

8×8 OffsetArray(::Matrix{Float64}, 1:8, 1:8) with eltype Float64 with indices 1:8×1:8:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.0
 0.0  0.0  0.0  0.0  0.0  0.5  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [491]:
eigvals(parent(G_star))

8-element Vector{Float64}:
 2.0420516199755806e-10
 0.0031532460821050005
 0.021078999899931557
 0.028023785658715203
 0.04457471801690229
 0.10348060401746054
 0.19670115116404868
 1.19751807124202

In [478]:
for i in 0:N
    for j in i+1:N
        P= G_star-(tr(G_star * D_mat_f(i, j, 𝐟′)) / tr( D_mat_f(i, j, 𝐟′)* D_mat_f(i, j, 𝐟′)) )*D_mat_f(i, j, 𝐟′)
        a=eigvals(parent(P))
        print(a)
    end
end

[-0.01862648626817136, 0.0007100523268583908, 0.020090060637288038, 0.027865937410023276, 0.042365937923955474, 0.13768864266871886, 0.19746629153913176, 1.194330387419861][0.0004240350533170398, 0.008442472402108752, 0.02101145147444199, 0.029752906310609624, 0.04206729321009044, 0.1042123631028535, 0.19746629153913212, 1.1985140105651122][0.0002475033203227554, 0.004575085336673269, 0.020220128300345863, 0.026159858036179658, 0.045182227887889545, 0.10988610249383682, 0.19746629153913173, 1.1981536267432853][0.0003175911709035529, 0.0022329778752468513, 0.020247900025947178, 0.028582189427271122, 0.04564162143706105, 0.10918244628460314, 0.19746629153913134, 1.1982198058975015][-4.776836512812716e-7, 0.0013078936521798263, 0.02211685958371016, 0.02822468293516794, 0.045449374260461216, 0.10913040075528212, 0.19746629153913167, 1.1981957986153833][0.00044957719068623015, 0.004696860079451054, 0.021016475688732987, 0.028177909106839896, 0.04250727471398908, 0.10924075125996072, 0.19746

In [481]:
i,j = 0 , 1
P = G_star-(tr(G_star * D_mat_f(i, j, 𝐟′)) / tr( D_mat_f(i, j, 𝐟′)* D_mat_f(i, j, 𝐟′)) )*D_mat_f(i, j, 𝐟′)

8×8 OffsetArray(::Matrix{Float64}, 1:8, 1:8) with eltype Float64 with indices 1:8×1:8:
  1.0          -1.66231e-16  -0.392123    …   0.0297827     0.0298196
 -1.66231e-16   0.197466      1.4088e-16      1.73483e-16  -1.90927e-16
 -0.392123      1.4088e-16    0.243213       -0.00559209   -0.00683579
 -0.182995     -5.08191e-17   0.0            -0.00387824   -0.00409928
 -0.0229093     5.14307e-18  -0.0168128      -0.00747646   -0.0019442
  0.0229106    -2.29321e-17  -0.0106426   …  -0.007407     -0.00327691
  0.0297827     1.73483e-16  -0.00559209      0.0157765    -0.00702726
  0.0298196    -1.90927e-16  -0.00683579     -0.00702726    0.0134867

In [482]:
eigvals(parent(P))

8-element Vector{Float64}:
 -0.01862648626817136
  0.0007100523268583908
  0.020090060637288038
  0.027865937410023276
  0.042365937923955474
  0.13768864266871886
  0.19746629153913176
  1.194330387419861

In [488]:
tr(G_star * D_mat_f(0, 1, 𝐟′))

0.03935220334899613

In [875]:
γ = OffsetArray(zeros(N+2, N+2), -1:N, -1:N)
for i in -1:N
    for j in -1:N
        #γ[i,j]=tr(G_star*( A_mat_f(i, j, α, 𝐟′, 𝐰)))
       γ[i,j]=F_star'*a_vec_f(i, j, 𝐟) + tr(G2*( A_mat_f(i, j, α, 𝐟′, 𝐰)+ (1/(2*L)) * C_mat_f(i, j, 𝐟′) ) )
    end
end

In [876]:
γ

5×5 OffsetArray(::Matrix{Float64}, -1:3, -1:3) with eltype Float64 with indices -1:3×-1:3:
  0.0         -2.09353e-9  -3.06018e-10   1.53595e-9   -1.64169e-8
 -0.153285     0.0          1.30725e-9   -0.147451     -0.218736
 -0.0663036   -0.0807707    0.0           6.93873e-9   -0.149314
 -5.61741e-6  -3.82892e-5  -3.25157e-5    0.0           8.53807e-9
  2.57412e-9  -1.01e-7     -1.45712e-8   -9.22805e-10   0.0

In [841]:
γ

5×5 OffsetArray(::Matrix{Float64}, -1:3, -1:3) with eltype Float64 with indices -1:3×-1:3:
  0.0        -6.20969e-9   3.48553e-9    2.06829e-8   1.85893e-8
 -0.144566    0.0         -6.42038e-10  -0.157572    -0.199801
 -0.0382624  -0.135773     0.0           1.59744e-8  -0.0585517
 -0.0272962  -0.161172    -0.0658091     0.0          2.04312e-8
 -0.0198215  -0.116929    -0.0477601    -4.21334e-5   0.0

In [837]:
γ

5×5 OffsetArray(::Matrix{Float64}, -1:3, -1:3) with eltype Float64 with indices -1:3×-1:3:
  0.0        -1.32919e-9   2.52794e-9   9.7571e-9    8.83829e-9
 -0.0818989   0.0          1.50134e-9  -0.0253078   -0.0382104
 -0.0565911  -0.0253059    0.0          7.56323e-9  -0.0129026
 -0.0436885  -0.0266638   -0.012896     0.0          9.46043e-9
 -0.0316963  -0.0193449   -0.00935799  -6.82773e-6   0.0

In [411]:
eigvals(parent(γ))

6-element Vector{Float64}:
 -0.2929813305956378
 -1.5259755789374353e-8
  1.6679593754197695e-5
  0.010588375681233133
  0.07293748653880101
  0.20943880404160498

In [471]:
function construct_grammian(N,L,G_star)
    G = OffsetArray(zeros(N+1, N+1), 0:N, 0:N)
    α=new_ogm(N)
    𝐰, 𝐟′, 𝐟 = data_generator_function(N, α,  L)
    
    for i in 0:N
        for j in 0:N
            G[i,j]= tr(G_star*D_mat_f(i, j, 𝐟′))
        end
    end
     
    return G
end

construct_grammian (generic function with 2 methods)

In [472]:
G=construct_grammian(N,L,G_star)

6×6 OffsetArray(::Matrix{Float64}, 0:5, 0:5) with eltype Float64 with indices 0:5×0:5:
  0.243213     0.0340847   -0.0168128   -0.0106426   -0.00559209  -0.00683579
  0.0340847    0.0743818   -0.00335002  -0.0117029   -0.00387824  -0.00409928
 -0.0168128   -0.00335002   0.0350735   -0.00493655  -0.00747646  -0.0019442
 -0.0106426   -0.0117029   -0.00493655   0.0224929   -0.007407    -0.00327691
 -0.00559209  -0.00387824  -0.00747646  -0.007407     0.0157765   -0.00702726
 -0.00683579  -0.00409928  -0.0019442   -0.00327691  -0.00702726   0.0134867

In [417]:
eigvals(parent(G))

5-element Vector{Float64}:
 0.0005887298948850303
 0.02780508538352686
 0.044499688676529016
 0.08137342911737269
 0.2950311129739603

In [81]:
 0.356761    0.0215865  -0.047764   -0.0199191
  0.0215865   0.0929622  -0.0217487  -0.0226351
 -0.047764   -0.0217487   0.041454   -0.0210429
 -0.0199191  -0.0226351  -0.0210429   0.0357417

5×5 OffsetArray(::Matrix{Float64}, 0:4, 0:4) with eltype Float64 with indices 0:4×0:4:
  0.0660946     5.61342e-15   5.73804e-16   3.91333e-15  -5.42786e-16
  5.61342e-15   0.0204245    -2.72843e-15  -9.45208e-16  -2.25148e-15
  5.73804e-16  -2.72843e-15   0.0104102    -2.37354e-16  -1.4124e-15
  3.91333e-15  -9.45208e-16  -2.37354e-16   0.00641661   -5.41688e-17
 -5.42786e-16  -2.25148e-15  -1.4124e-15   -5.41688e-17   0.00651773

In [75]:
for i in 1:N
    print(sum(α[i,k]*G[k,3] for k in 0:i-1))
end

-0.03239537075429325-0.06806000978154811-0.09761326046863925-0.06388778736810996

In [95]:
for i in 0:N
    for j in 0:N
        print(-F_star[i+2]+F_star[j+2]+(1/2)*G[i,i]+(1/2)*G[j,j])
    end
end

0.066094623539094012.7220269701450084e-9-0.02042448748858703-0.030834624591446095-0.0372512223919009150.086519123306706330.020424502489639281.2279025281801415e-8-0.010410124823833784-0.01682672262428860.096929263085601870.030834642268534830.0104101520579208281.4955061762318006e-8-0.0064165828453930520.103345862119912590.0372512413028455450.0168267510922315460.00641661398937248051.6188917666918462e-80.109863574562624540.04376895374555750.0233444635349434950.012934326432084430.006517728631629615

In [94]:
F_star[7]

LoadError: BoundsError: attempt to access 6-element Vector{Float64} at index [7]

In [20]:
##dual
function solve_inner_dual(N, L, α, R; 
        show_output = :off, 
        ϵ_tol = 1e-6, 
        objective_type = :default, 
        obj_val_upper_bound = default_obj_val_upper_bound)

    𝐰, 𝐟′, 𝐟 = data_generator_function(N, α, L)

    dim_Z=N+3
    idx_set_λ = index_set_constructor(N)

    #model
    model_dual = Model(optimizer_with_attributes(Mosek.Optimizer))

    #variables
    
    @variable(model_dual, λ[idx_set_λ] >= 0)
    @variable(model_dual, ν >= 0)
    @variable(model_dual, Z[1:dim_Z, 1:dim_Z], PSD)
    
    #objectives

    if objective_type == :default
        # objective 0: define default objective
        @objective(model_dual, Min,  ν*R^2)
        @constraint(model_dual, ν*R^2 <= obj_val_upper_bound)

    elseif objective_type == :sparsify_λ
        # objective 1: sparse dual solution in λ
        @objective(model_dual, Min, sum(λ[i_j] for i_j in idx_set_λ)  )
        @constraint(model_dual, ν*R^2 <= obj_val_upper_bound)
           
    elseif objective_type == :sparsify_Z
        # objective 3: sparse dual solution in Z
        @objective(model_dual, Min, tr(Z) )
        @constraint(model_dual, ν*R^2 <= obj_val_upper_bound)
        
    elseif objective_type == :find_M_λ
        # objective 4: to compute M_λ
        @objective(model_dual, Max, sum(λ[i_j] for i_j in idx_set_λ))
        @constraint(model_dual, ν*R^2 <= obj_val_upper_bound)

    elseif objective_type == :find_M_Z
        # objective 6: to compute M_Z
        @objective(model_dual, Max, tr(Z))
        @constraint(model_dual, ν*R^2 <= obj_val_upper_bound)
    end
    
    #constraints

    @constraint(model_dual, -a_vec_f(-1, N, 𝐟) +
    sum( λ[i_j_λ]*a_vec_f(i_j_λ.i, i_j_λ.j, 𝐟) for i_j_λ in idx_set_λ).==0)

    @constraint(model_dual, 
    sum( λ[i_j_λ]*( A_mat_f(i_j_λ.i, i_j_λ.j, α,  𝐟′, 𝐰) + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) for i_j_λ in idx_set_λ)
    + ν*B_mat(0, -1, α, 𝐰).== Z)
    
    
    #optimize
    
    if show_output == :off
        set_silent(model_dual)
    end
    
    optimize!(model_dual)

    if termination_status(model_dual) != MOI.OPTIMAL
        @info "💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀"
        @error "dual did not reach optimality;  termination status = " termination_status(model_dual)
    end

    
    #store and return
    λ_opt = value.(λ)
    ν_opt = value.(ν)
    Z_opt = value.(Z)
    L_cholesky_opt =  compute_pivoted_cholesky_L_mat(Z_opt)
    obj_val_dual = objective_value(model_dual)
          
    if norm(Z_opt - L_cholesky_opt*L_cholesky_opt', Inf) >  ϵ_tol
        @warn "||Z - L*L^T|| = $(norm(Z_opt - L_cholesky_opt*L_cholesky_opt', Inf))"
    end
   
    #some heuristics (For now, focus on upper bounds and effective indices)

    idx_set_λ_effective = effective_index_set_finder(λ_opt, ϵ_tol)
    l1_norm_λ = sum(λ_opt)
    tr_Z = tr(Z_opt)
    sparse_sol_obj_val = l1_norm_λ 
    
    @info "performance measure of dual is $(obj_val_dual)"

    return obj_val_dual, l1_norm_λ, tr_Z, sparse_sol_obj_val, λ_opt, ν_opt, Z_opt, L_cholesky_opt, idx_set_λ_effective
    
end


## BnB-PEP solver main function
# =============================

function BnB_PEP_solver(N, L, R,
    # solutions to warm-start
    obj_val_dual, λ_ws, ν_ws, Z_ws, L_cholesky_ws, 
    α_ws, idx_set_λ_effective_ws, 
    # upper bounds
    M_λ, M_ν, M_Z, M_L_cholesky ;
    # options    
    solution_type = :find_locally_optimal, # other option :find_globally_optimal 
    show_output = :off, # other option :on
    reduce_index_set_for_λ = :for_warm_start_only,
    # on  : force not effective sets 0 
    # off , do nothing 
    # for_warm_start_only , =off but warm start from effective indices
    reduce_index_set_for_L_cholesky = :off, 
    positive_step_size = :off, 
    quadratic_equality_modeling = :through_ϵ,  #no other options yet
    ϵ_tol = 1e-6, 
    ϵ_tol_Cholesky = 0.0005, 
    global_lower_bound = 0.0, 
    local_solver = :ipopt, # options are :knitro and :ipopt
    fix_diagonals =:on
    )

    dim_Z = N+3

    # model
    # =====

    if solution_type == :find_globally_optimal
            
        BnB_PEP_model =  Model(Gurobi.Optimizer)

        set_optimizer_attribute(BnB_PEP_model, "NonConvex", 2)
        #NonConvex = 2 for nonconvex spatial branching

        set_optimizer_attribute(BnB_PEP_model, "MIPFocus", 2)
            
        #The MIPFocus parameter allows you to modify your high-level solution strategy, 
        #depending on your goals. By default, the Gurobi MIP solver strikes 
        #a balance between finding new feasible solutions and proving that the current solution is optimal. 
        #If you are more interested in finding feasible solutions quickly, you can select MIPFocus=1. 
        #If you believe the solver is having no trouble finding good quality solutions, 
        #and wish to focus more attention on proving optimality, select MIPFocus=2. 
        #If the best objective bound is moving very slowly (or not at all)
        #, you may want to try MIPFocus=3 to focus on the bound.

        set_optimizer_attribute(BnB_PEP_model, "MIPGap", 1e-2) 
        #The MIP solver will terminate (with an optimal result) when the gap between 
        #the lower and upper objective bound is less than 
        #MIPGap times the absolute value of the incumbent objective value.

    elseif solution_type == :find_locally_optimal

        if local_solver == :ipopt
        
            BnB_PEP_model = Model(Ipopt.Optimizer)

        elseif local_solver == :knitro 

            BnB_PEP_model = Model(
                optimizer_with_attributes(
                KNITRO.Optimizer,
                "convex" => 0,
                "strat_warm_start" => 1,
                # the last settings below are for larger N
                "honorbnds" => 1
                # "bar_feasmodetol" => 1e-3,
                # "feastol" => 1e-4,
                # "infeastol" => 1e-12,
                # "opttol" => 1e-4
                )
            )

        end
        
    end

    #variables
    @info "adding variables"
            
    #stepsizes α,β,ϕ,ψ
            
    if positive_step_size == :off
        @variable(BnB_PEP_model, α[i=1:N, j=0:N-1])
    elseif positive_step_size == :on
        @variable(BnB_PEP_model, α[i=1:N, j=0:N-1] >= 0)
    end


    #λ, τ

    if reduce_index_set_for_λ == :off
        idx_set_λ = index_set_constructor(N)
        @variable(BnB_PEP_model, λ[idx_set_λ] >= 0)
    elseif reduce_index_set_for_λ == :on
        idx_set_λ = idx_set_λ_effective_ws
        @variable(BnB_PEP_model, λ[idx_set_λ] >= 0)
    elseif reduce_index_set_for_λ == :for_warm_start_only   
        idx_set_λ = index_set_constructor(N)
        idx_set_λ_ws = idx_set_λ_effective_ws    
        @variable(BnB_PEP_model, λ[idx_set_λ] >= 0) #define λ for all index sets
    end
        
    #γ,ν,Z,cholskey
    @variable(BnB_PEP_model, ν >= 0)
    @variable(BnB_PEP_model, Z[1:dim_Z, 1:dim_Z], Symmetric)
    @variable(BnB_PEP_model, L_cholesky[1:dim_Z, 1:dim_Z])
    
      
    
    #objective
    @info "adding objective"

    @objective(BnB_PEP_model, Min,ν*R^2)  
    #upper,lower bound
    @constraint(BnB_PEP_model,  ν*R^2 <= 1.2*obj_val_dual) 
    @constraint(BnB_PEP_model,  ν*R^2 >= global_lower_bound)
    
    @info "adding data generator"
    𝐰, 𝐟′, 𝐟 = data_generator_function(N, α,  L)
    
    
                 
    #constraints
    @info "adding constraints"
      
#constraints

    @constraint(BnB_PEP_model, -a_vec_f(-1, N, 𝐟) +
    sum( λ[i_j_λ]*a_vec_f(i_j_λ.i, i_j_λ.j, 𝐟) for i_j_λ in idx_set_λ).==0)

    @constraint(BnB_PEP_model, 
    vectorize(sum( λ[i_j_λ]*( A_mat_f(i_j_λ.i, i_j_λ.j, α,  𝐟′, 𝐰) + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) for i_j_λ in idx_set_λ)
    + ν*B_mat(0, -1, α, 𝐰)-Z- ϵ_tol*ones(dim_Z,dim_Z),SymmetricMatrixShape(dim_Z)).<=0  ) 
    
    @constraint(BnB_PEP_model, 
    vectorize(sum( λ[i_j_λ]*( A_mat_f(i_j_λ.i, i_j_λ.j, α,  𝐟′, 𝐰) + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) for i_j_λ in idx_set_λ)
    + ν*B_mat(0, -1, α, 𝐰)-Z+ ϵ_tol*ones(dim_Z,dim_Z),SymmetricMatrixShape(dim_Z)).>=0  ) 
    
            
    # Cholesky things

    for i in 1:dim_Z
        for j in 1:dim_Z
            if i < j
                fix(L_cholesky[i,j], 0; force = true)
            end
        end
    end

    for i in 1:dim_Z
        @constraint(BnB_PEP_model, L_cholesky[i,i] >= 0)
    end

    @constraint(BnB_PEP_model, vectorize(Z - (L_cholesky * L_cholesky') - ϵ_tol*ones(dim_Z,dim_Z), 
                    SymmetricMatrixShape(dim_Z)) .<= 0)
    @constraint(BnB_PEP_model, vectorize(Z - (L_cholesky * L_cholesky') + ϵ_tol*ones(dim_Z,dim_Z), 
                    SymmetricMatrixShape(dim_Z)) .>= 0)
                        
    if reduce_index_set_for_L_cholesky == :on
        zeroindex = zero_finder_cholesky(L_cholesky_ws; ϵ_tol = ϵ_tol_Cholesky)
        for k in 1:length(zeroindex)
            fix(L_cholesky[CartesianIndex(zeroindex[k])], 0; force = true)
        end
    end

    # implicit constraints
            
    for i in 1:dim_Z
        @constraint(BnB_PEP_model, Z[i,i] >= 0)
    end
            
    for i in 1:dim_Z
        for j in 1:dim_Z
            if i != j
                @constraint(BnB_PEP_model, Z[i,j] <= (0.5*(Z[i,i] + Z[j,j])))
                @constraint(BnB_PEP_model, -0.5*(Z[i,i] + Z[j,j]) <= Z[i,j])
            end
        end
    end
    
    for i in 0:N-1
        for j in 0:N-1
            if j > i
                fix(α[i+1,j], 0; force = true)
            end
        end
    end

            
    #give bounds
    @info "adding upper bounds"

    set_upper_bound.(λ, M_λ)
    set_upper_bound(ν, M_ν)
            
    for i in 1:dim_Z
        for j in 1:dim_Z
            set_lower_bound(Z[i,j], -M_Z)
            set_upper_bound(Z[i,j], M_Z)
        end
    end
    
    for i in 1:dim_Z
        set_upper_bound(L_cholesky[i,i], M_L_cholesky)
    end

    for i in 1:dim_Z
        for j in 1:dim_Z
            if i > j
                set_lower_bound(L_cholesky[i,j], -M_L_cholesky)
                set_upper_bound(L_cholesky[i,j], M_L_cholesky)
            end
        end
    end
    
      
    #warm start
    @info "setting warm-start values"
            
    for i in 0:N-1
        for j in 0:i
            set_start_value(α[i+1,j], α_ws[i+1,j])
        end
    end
    

    if reduce_index_set_for_λ == :for_warm_start_only
                
        for i_j_λ in idx_set_λ_ws
            set_start_value(λ[i_j_λ], λ_ws[i_j_λ])
        end

    else
                
        for i_j_λ in idx_set_λ
            set_start_value(λ[i_j_λ], λ_ws[i_j_λ])
        end
                
    end

    set_start_value(ν, ν_ws)
            
    for i in 1:dim_Z
        for j in 1:dim_Z
            set_start_value(Z[i,j], Z_ws[i,j])
            set_start_value(L_cholesky[i,j], L_cholesky_ws[i,j])
        end
    end
    
     
    if fix_diagonals ==:on
        a=new_ogm(N)
        for i in 1:N-1
            @constraint(BnB_PEP_model, α[N,i-1]==α[N-1,i-1])
            fix(α[N,N-1],1, force=true )
        end  
    end
       
    # optimize
    @info "ready to optimize"
    
    if show_output == :off
        set_silent(BnB_PEP_model)
    end

    # Shuvo start

    optimize!(BnB_PEP_model)

    @info "BnB_PEP_model has termination status = " termination_status(BnB_PEP_model)

    # store and return   
    λ_opt = value.(λ)
    ν_opt = value.(ν)
    Z_opt = value.(Z)
    L_cholesky_opt = value.(L_cholesky)

    α_opt = value.(α)
    idx_set_λ_opt_effective = effective_index_set_finder(λ_opt,ϵ_tol)   
 
    obj_val = ν_opt*R^2 
    
    @info "Objective value = $(obj_val)"
    
    
    if norm(Z_opt - L_cholesky_opt*L_cholesky_opt', Inf) > ϵ_tol
        @warn "||Z - L_cholesky*L_cholesky^T|| = $(norm(Z_opt -  L_cholesky_opt*L_cholesky_opt', Inf))"
    end
 
        
    return obj_val, λ_opt, ν_opt, Z_opt, L_cholesky_opt,
    α_opt, idx_set_λ_opt_effective
    
    
    @info "α = $(α_opt)" 
    
    @info "effective set for λ = $(idx_set_λ_opt_effective)" 

end

     

BnB_PEP_solver (generic function with 1 method)

In [21]:
function data_generator_function_OGM(N,L)

    dim_𝐰 = N+2
    dim_𝐟′ = N+2
    dim_𝐟 = N+1
    
    𝐰_star = zeros(dim_𝐰, 1) #optimum
    𝐰_0 = e_i(dim_𝐰, 1) #starting point
    θ=get_theta(N)

    #create gradients
    
    𝐟′ = OffsetArray(zeros(dim_𝐟′, N+2), 1:dim_𝐟′, -1:N) 

    # fill the elements of 𝐟′

    for i in 0:N
        𝐟′[:,i] = e_i(dim_𝐟′, i+2)
    end
    
    #create function evaluations

    𝐟 = OffsetArray(zeros(dim_𝐟, N+2), 1:dim_𝐟, -1:N)

    # fill elements of 𝐟

    for i in 0:N
        𝐟[:,i] = e_i(dim_𝐟, i+1)
    end   

    #make 𝐰
    
    𝐰 = OffsetArray(Matrix{Any}(undef, dim_𝐰, N+2), 1:dim_𝐰, -1:N)
    𝐳 = OffsetArray(Matrix{Any}(undef, dim_𝐰, N+2), 1:dim_𝐰, -1:N)

    𝐰[:,-1] = 𝐰_star
    𝐳[:,-1] = 𝐰_star

    𝐰[:, 0] = 𝐰_0
    𝐳[:,0] = 𝐰_0
    
    α=new_ogm(N)
    #α=construct_GD_stepsizes(N)
    for i in 0:N-1
        𝐳[:, i+1] = 𝐳[:, i] - ((2*θ[i])/L)*𝐟′[:,i]
        𝐰[:, i+1] = 𝐰_0 - (1/L)*sum(α[i+1,j]*𝐟′[:,j] for j in 0:i) 
    end

    return 𝐰, 𝐟′, 𝐟 , 𝐳

end

data_generator_function_OGM (generic function with 1 method)

In [22]:
##dual
function solve_OGM_lya(N, L, F_star, G_star, λ_opt ; 
        show_output = :off, 
        ϵ_tol = 1e-6, )
    
    
    α=new_ogm(N)
    𝐰, 𝐟′, 𝐟 = data_generator_function(N, α, L)
    θ=get_theta(N)
    
    #model
    #model_dual = Model(optimizer_with_attributes(Mosek.Optimizer))
    model_dual = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model_dual, "NonConvex", 2)
    idx_set_λ = index_set_constructor(N)
    
    U1=sum( λ_opt[i_j_λ]* (F_star'*a_vec_f(i_j_λ.i, i_j_λ.j, 𝐟) + tr(G_star*( A_mat_f(i_j_λ.i, i_j_λ.j, α, 𝐟′, 𝐰)
            + (1/(2*L)) * C_mat_f(i_j_λ.i, i_j_λ.j, 𝐟′) ) )) for i_j_λ in idx_set_λ)
    
    X = (L/(2*θ[N]^2))* tr(G_star*⊙(𝐰[:,0]-𝐰[:,-1], 𝐰[:,0]-𝐰[:,-1]))
    
    
    #variables

    @variable(model_dual, u[0:N])
    @variable(model_dual, w>=0)
        
    obj= (F_star'a_vec_f(-1, N, 𝐟))
    
    z= sum( u[i]* 𝐟′[:,i] for i in 0:N)
    
    U2= (L/(2*θ[N]^2))*tr(G_star*⊙(w*𝐰[:,0]-z, w*𝐰[:,0]-z))

    @constraint(model_dual, w^2==1)
    
    #objectives

    @objective(model_dual, Min, U2+obj-U1-X)

    
    #optimize
    
    if show_output == :off
        set_silent(model_dual)
    end
    
    optimize!(model_dual)

    if termination_status(model_dual) != MOI.OPTIMAL
        @info "💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀💀"
        @error "dual did not reach optimality;  termination status = " termination_status(model_dual)
    end
    #store and return
    u_opt = value.(u)
    w_opt = value.(w)
    obj_val_dual = objective_value(model_dual)
          
    @info "performance measure of dual is $(obj_val_dual)"

    return obj_val_dual, u_opt , w_opt
    
end


solve_OGM_lya (generic function with 1 method)

In [23]:
N=4
L=1
R=1

α=new_ogm(N)

obj_val_primal, G_star, F_star= solve_inner_primal(N, L, α, R; show_output = :off)


obj_val_dual, l1_norm_λ_dual, tr_Z_dual, 
sparse_sol_obj_val, λ_opt, ν_ws, 
Z_ws, L_cholesky_ws, idx_set_λ_effective_ws = solve_inner_dual(N, L, α,  R; show_output = :off, 
    ϵ_tol = 1e-6, objective_type = :default, obj_val_upper_bound = 1.1*obj_val_primal)


solve_OGM_lya(N, L, F_star, G_star, λ_opt ; show_output = :off, ϵ_tol = 1e-6, )

┌ Info: performance measure of primal is 0.02558397015264716
└ @ Main In[1]:378


LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 6 and 7")

In [24]:
θ=get_theta(N)

5-element OffsetArray(::Vector{Float64}, 0:4) with eltype Float64 with indices 0:4:
 1.0
 1.618033988749895
 2.193527085331054
 2.749791340120445
 4.420804104823752

In [25]:
2*θ

5-element OffsetArray(::Vector{Float64}, 0:4) with eltype Float64 with indices 0:4:
 2.0
 3.23606797749979
 4.387054170662108
 5.49958268024089
 8.841608209647504

In [26]:
λ_opt

LoadError: UndefVarError: λ_opt not defined

In [27]:
function get_theta_NAG(N)
    theta=OffsetArray(zeros(N+1), 0:N)
    theta[0]=1
    for i in 1:N
        theta[i]=(1+sqrt(1+4*theta[i-1]^2))/2
    end
    return theta
end


θ=get_theta_NAG(3)
1/(2*θ[N-1]^2)

0.0661257368537568

In [28]:
1/ 0.04055411156359865

24.658412216274503

In [29]:
## test
N=5
L=1
R=1

α=new_ogm(N)

#α=construct_GD_stepsizes(N)
obj_val_primal, G_star_primal, F_star_primal= solve_inner_primal(N, L, α, R; show_output = :off)


obj_val_dual, l1_norm_λ_dual, tr_Z_dual, 
sparse_sol_obj_val, λ_ws, ν_ws, 
Z_ws, L_cholesky_ws, idx_set_λ_effective_ws = solve_inner_dual(N, L, α,  R; show_output = :off, 
    ϵ_tol = 1e-6, objective_type = :default, obj_val_upper_bound = 1.1*obj_val_primal)

@info "primal-dual gap for stage 1 = $(obj_val_primal-obj_val_dual)"  


## Impose bound

dim_Z_ws, _ = size(Z_ws)

big_M_G = maximum(Z_ws[i,i] for i in 1:dim_Z_ws)

big_M_λ = maximum(λ_ws)

big_M = 5*maximum([big_M_G; big_M_λ])

M_λ,  M_ν, M_Z, M_L_cholesky = big_M , big_M *ν_ws, big_M, sqrt(big_M)

α_ws = α

## Find locally optimal solution

obj_val, λ_opt, ν_opt, Z_opt, L_cholesky_opt, 
α_opt,idx_set_λ_opt_effective = BnB_PEP_solver(N, L, R,
    # solutions to warm-start
    obj_val_dual, λ_ws, ν_ws, Z_ws, L_cholesky_ws, 
    α_ws ,  idx_set_λ_effective_ws,
    # upper bounds
    M_λ, M_ν, M_Z, M_L_cholesky ;
    # options    
    solution_type = :find_globally_optimal, # other option :find_globally_optimal 
    show_output = :off, # other option :on
    reduce_index_set_for_λ = :on, #:for_warm_start_only,
    # on  : force not effective sets 0 
    # off , do nothing 
    # for_warm_start_only , =off but warm start from effective indices
    reduce_index_set_for_L_cholesky = :on, 
    positive_step_size = :on, 
    quadratic_equality_modeling = :through_ϵ,  #no other options yet
    ϵ_tol = 1e-6, 
    ϵ_tol_Cholesky = 0.0005, 
    global_lower_bound = 0.0, 
    local_solver = :ipopt,
    fix_diagonals = :off
    )

## Display a few important info:

@show obj_val
##
@show α_opt          

┌ Info: performance measure of primal is 0.01858816102062284
└ @ Main In[1]:378


LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 7 and 8")

In [30]:
function construct_hOGM_stepsizes(N) #reference from OGM paper 
    α=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    h=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    theta=OffsetArray(zeros(N+1), 0:N)
    theta[0]=1
    for i in 1:N-1
        theta[i]=(1+sqrt(1+4*theta[i-1]^2))/2
    end
    theta[N]=(1+sqrt(1+8*theta[N-1]^2))/2


    for i in 0:N-1
        for k in 0:i-1
            if k+1<=i
                h[i+1,k]=(1/theta[i+1])*(2*theta[k]-sum(h[l,k] for l in k+1:i))
            end
        end
        h[i+1,i]=1+(2*theta[i]-1)/theta[i+1]
    end
    
    return h
end

function construct_rOGM_stepsizes(N) #reference from OGM paper 
    r=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    h=construct_hOGM_stepsizes(N)
    λ=get_lambda_OGM(N)
    τ=get_tau_OGM(N)
    
    for i in 1:N
        for j in 0:i-1
            r[i,j]=λ[i]*h[i,j]+τ[i]*sum(h[l,j] for l in j+1:i)
        end
    end

    return r
end

function construct_hhOGM_stepsizes(N) #reference from OGM paper 
    α=OffsetArray(zeros(N,N), 1:N, 0:N-1)
    r=construct_rOGM_stepsizes(N)
    λ=get_lambda_OGM(N)
    τ=get_tau_OGM(N)
    
    for i in 1:N
        α[i,i-1]=r[i,i-1]/(τ[i]+λ[i])
        for k in 0:i-2
            α[i,k]=(r[i,k]-τ[i]*sum(α[j,k] for j in k+1:i-1))/(τ[i]+λ[i])
        end
    end

    return α
end

construct_hhOGM_stepsizes (generic function with 1 method)

In [31]:
function get_theta_guler(N,diag)
    k=OffsetArray(zeros(N), 0:N-1)
    for i in 0:N-1
        k[i]=diag[i+1]/diag[1]
    end
    θ=OffsetArray(zeros(N), 0:N-1)
    θ[0]=1
    for i in 1:N-1
        θ[i]=(k[i]+sqrt(k[i]^2+(4*k[i]/k[i-1])*θ[i-1]^2))/2
    end
    return θ
end


function get_guler_stepsize(N,diag)
    k=OffsetArray(zeros(N), 0:N-1)
    for i in 0:N-1
        k[i]=diag[i+1]/diag[1]
    end
    θ=get_theta_guler(N,diag)
    ϕ = OffsetArray(zeros(N, N), 1:N, 0:N-1)
    
    
    for i in 1:N
        ϕ[i,i-1]=diag[i]
    end
    
    if N>=2
        for i in 2:N
            for j in 0:i-2
                 ϕ[i,j]=diag[i]*(2*θ[i-1]*θ[j]+(ϕ[i-1,j]*θ[i-2]^2)/(diag[i-1]))/(θ[i-1]^2)
            end
        end
    end
         
    return ϕ    
end

get_guler_stepsize (generic function with 1 method)